In [199]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [200]:
import pyarrow as pa
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from source import data_import

In [201]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler

In [202]:
import tensorflow as tf
from tensorflow.keras.utils import plot_model

import keras 
from keras.models import Sequential # intitialize the ANN
from keras.layers import Dense, Activation, Dropout 

In [203]:
data = np.load('data/feature_extract_reshaped_all.npy')
data.shape

(439, 30, 71)

In [204]:
# Split data into train and test set

train_split_ratio = int((data.shape[0])*0.8)  # the first dimensio is epoch, and we are splitting the epochs between train anf test
train_data = data[:train_split_ratio,:,:]
test_data = data[train_split_ratio:,:,:]
train_data.shape, test_data.shape

((351, 30, 71), (88, 30, 71))

In [205]:
target =train_data[:,:,-1]
target.shape

(351, 30)

In [206]:
features =train_data[:,:,:-1]
features.shape


(351, 30, 70)

In [207]:

X_train = train_data[:,:,:-1]
y_train = train_data[:,:,-1]
X_test = test_data[:,:,:-1]
y_test = test_data[:,:,-1]


In [208]:
X_train.shape

(351, 30, 70)

In [209]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 1., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [210]:
inputs = tf.constant(X_train, dtype=tf.float32)
lstm = tf.keras.layers.LSTM(4)
output = lstm(inputs)
print(output.shape)



(351, 4)


In [211]:
inputs = tf.constant(X_train, dtype=tf.float32)
lstm = tf.keras.layers.LSTM(4, return_sequences=True, return_state=True)
whole_seq_output, final_memory_state, final_carry_state = lstm(inputs)
print(whole_seq_output.shape)
print(final_memory_state.shape)
print(final_carry_state.shape)


(351, 30, 4)
(351, 4)
(351, 4)


In [212]:
inputs.shape

TensorShape([351, 30, 70])

In [213]:
shape=X_train.shape[1:]
shape

y_train_1d = y_train.max(axis=1)
y_train_1d

array([0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0.,
       1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0.,
       1., 0., 1., 0., 0.

In [214]:
positions = np.isnan(y_train)
np.any(positions)



False

In [215]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Input

model = Sequential()

model.add(Input(shape=X_train.shape[1:]))

# Recurrent layer
model.add(LSTM(64, return_sequences=False, 
               dropout=0.1, recurrent_dropout=0.1))

# Fully connected layer
model.add(Dense(64, activation='relu'))

# Dropout for regularization
model.add(Dropout(0.5))

# Output layer
model.add(Dense(1, activation='sigmoid'))

# Compile the model
from tensorflow.keras.optimizers import Adam

learning_rate = 0.001
adam_optimizer = Adam(learning_rate=learning_rate)


model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [216]:
history = model.fit(X_train,  y_train_1d, 
                    # batch_size=2048, 
                    epochs=10)

Epoch 1/10
11/11 [==============================] - 6s 28ms/step - loss: 0.4928 - accuracy: 0.8575
Epoch 2/10
11/11 [==============================] - 0s 19ms/step - loss: 0.4326 - accuracy: 0.8604
Epoch 3/10
11/11 [==============================] - 0s 16ms/step - loss: 0.4083 - accuracy: 0.8604
Epoch 4/10
11/11 [==============================] - 0s 13ms/step - loss: 0.4320 - accuracy: 0.8604
Epoch 5/10
11/11 [==============================] - 0s 12ms/step - loss: 0.4215 - accuracy: 0.8604
Epoch 6/10
11/11 [==============================] - 0s 13ms/step - loss: 0.4087 - accuracy: 0.8604
Epoch 7/10
11/11 [==============================] - 0s 12ms/step - loss: 0.4314 - accuracy: 0.8604
Epoch 8/10
11/11 [==============================] - 0s 12ms/step - loss: 0.4098 - accuracy: 0.8604
Epoch 9/10
11/11 [==============================] - 0s 11ms/step - loss: 0.4315 - accuracy: 0.8604
Epoch 10/10
11/11 [==============================] - 0s 12ms/step - loss: 0.4024 - accuracy: 0.8604


In [217]:
y_pred = model.predict(X_test)

3/3 [==============================] - 0s 5ms/step


In [218]:
y_pred

array([[0.14237745],
       [0.13149945],
       [0.14296146],
       [0.1331521 ],
       [0.13313702],
       [0.1301522 ],
       [0.13027324],
       [0.12999949],
       [0.13377638],
       [0.1437609 ],
       [0.14166959],
       [0.13573408],
       [0.16858   ],
       [0.13799025],
       [0.13615131],
       [0.1377168 ],
       [0.13006574],
       [0.13114205],
       [0.13068748],
       [0.1323889 ],
       [0.1392377 ],
       [0.13349335],
       [0.13514687],
       [0.13559957],
       [0.14658883],
       [0.13928244],
       [0.15004362],
       [0.1458116 ],
       [0.15559104],
       [0.14691241],
       [0.13859473],
       [0.13941905],
       [0.1599555 ],
       [0.15309758],
       [0.16385478],
       [0.13886087],
       [0.13747099],
       [0.1345817 ],
       [0.14857478],
       [0.16089748],
       [0.15204145],
       [0.14465776],
       [0.14352669],
       [0.1493115 ],
       [0.14560072],
       [0.1569815 ],
       [0.14645132],
       [0.143

In [219]:
# from keras.callbacks import EarlyStopping, ModelCheckpoint

# # Create callbacks
# callbacks = EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
history = model.fit(X_train,  y_train.astype(int), 
                    # batch_size=2048, 
                    epochs=10)

Epoch 1/10
12/12 [==============================] - 0s 12ms/step - loss: nan - accuracy: 0.8612
Epoch 2/10
12/12 [==============================] - 0s 10ms/step - loss: nan - accuracy: 0.8612
Epoch 3/10
12/12 [==============================] - 0s 10ms/step - loss: nan - accuracy: 0.8612
Epoch 4/10
12/12 [==============================] - 0s 10ms/step - loss: nan - accuracy: 0.8612
Epoch 5/10
12/12 [==============================] - 0s 10ms/step - loss: nan - accuracy: 0.8612
Epoch 6/10
12/12 [==============================] - 0s 10ms/step - loss: nan - accuracy: 0.8612
Epoch 7/10
12/12 [==============================] - 0s 10ms/step - loss: nan - accuracy: 0.8612
Epoch 8/10
12/12 [==============================] - 0s 10ms/step - loss: nan - accuracy: 0.8612
Epoch 9/10
12/12 [==============================] - 0s 10ms/step - loss: nan - accuracy: 0.8612
Epoch 10/10
12/12 [==============================] - 0s 10ms/step - loss: nan - accuracy: 0.8612


In [ ]:
# def make_model(input_shape):
#     input_layer = keras.layers.Input(input_shape)

#     conv1 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(input_layer)
#     conv1 = keras.layers.BatchNormalization()(conv1)
#     conv1 = keras.layers.ReLU()(conv1)

#     conv2 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv1)
#     conv2 = keras.layers.BatchNormalization()(conv2)
#     conv2 = keras.layers.ReLU()(conv2)

#     conv3 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv2)
#     conv3 = keras.layers.BatchNormalization()(conv3)
#     conv3 = keras.layers.ReLU()(conv3)

#     gap = keras.layers.GlobalAveragePooling1D()(conv3)

#     output_layer = keras.layers.Dense(num_classes, activation="softmax")(gap)

#     return keras.models.Model(inputs=input_layer, outputs=output_layer)


In [ ]:
# model = make_model(input_shape=X_train.shape[1:])
# keras.utils.plot_model(model, show_shapes=True)

In [ ]:
# epochs = 500
# batch_size = 32

# callbacks = [
#     keras.callbacks.ModelCheckpoint(
#         "best_model.h5", save_best_only=True, monitor="val_loss"
#     ),
#     keras.callbacks.ReduceLROnPlateau(
#         monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001
#     ),
#     keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1),
# ]
# model.compile(
#     optimizer="adam",
#     loss="sparse_categorical_crossentropy",
#     metrics=["sparse_categorical_accuracy"],
# )
# history = model.fit(
#     X_train,
#     y_train,
#     batch_size=batch_size,
#     epochs=epochs,
#     callbacks=callbacks,
#     #validation_split=0.2,
#     verbose=1,


inputs: A 3D tensor with shape [batch, timesteps, feature].

In [ ]:
# # Define a sequential model
# model = keras.Sequential()

# # Add an input layer with an input shape of (time_steps, input_features)
# model.add(layers.InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))

# # Add a SimpleRNN layer with 32 units (neurons)
# model.add(layers.SimpleRNN(32, activation='tanh'))

# # Add the output layer with 1 neuron and sigmoid activation for binary classification
# model.add(layers.Dense(30, activation='sigmoid'))

# # Compile the model with binary cross-entropy loss for binary classification
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
X_train.shape

(353, 30, 70)

In [ ]:
# Assuming X_train and y_train are your training data
# model.fit(X_train, y_train, epochs=100, batch_size=30)

In [ ]:
y_pred = model.predict(X_test)

3/3 [==============================] - 0s 4ms/step


In [221]:
y_pred

array([[0.14237745],
       [0.13149945],
       [0.14296146],
       [0.1331521 ],
       [0.13313702],
       [0.1301522 ],
       [0.13027324],
       [0.12999949],
       [0.13377638],
       [0.1437609 ],
       [0.14166959],
       [0.13573408],
       [0.16858   ],
       [0.13799025],
       [0.13615131],
       [0.1377168 ],
       [0.13006574],
       [0.13114205],
       [0.13068748],
       [0.1323889 ],
       [0.1392377 ],
       [0.13349335],
       [0.13514687],
       [0.13559957],
       [0.14658883],
       [0.13928244],
       [0.15004362],
       [0.1458116 ],
       [0.15559104],
       [0.14691241],
       [0.13859473],
       [0.13941905],
       [0.1599555 ],
       [0.15309758],
       [0.16385478],
       [0.13886087],
       [0.13747099],
       [0.1345817 ],
       [0.14857478],
       [0.16089748],
       [0.15204145],
       [0.14465776],
       [0.14352669],
       [0.1493115 ],
       [0.14560072],
       [0.1569815 ],
       [0.14645132],
       [0.143

In [226]:
from sklearn.metrics import precision_score, recall_score
y_pred_c = y_pred.round().astype(int)
precision = precision_score(y_test, y_pred_c, average='macro')
recall = recall_score(y_test, y_pred_c, average='macro')

loss, accuracy = model.evaluate(X_test, y_test)

print('Loss:', loss)
print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)

ValueError: Classification metrics can't handle a mix of multilabel-indicator and binary targets